Quoting: https://github.com/romanmichaelpaolucci/Quant-Guild-Library/blob/main/2025%20Video%20Lectures/2.%20Control%20Variates%20for%20Variance%20Reduction/Control%20Variates%20for%20Variance%20Reduction.ipynb
Control variates method reduces the variance for monte carlo methods. Using errors of a known quantity to reduce error of an esstimate of an unknown quantity. </br>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Parameters
S0 = 100         # Initial price
K = 100           # Strike price (set below S0 for more meaningful payoffs)
T = 252            # Time to maturity
mu = .1        # Drift
sigma = 0.3      # Volatility
n_simulations = 10000  # Number of simulations
n_steps = 252    # Time steps for ABM

# Simulating ABM paths
dt = T / n_steps
time_grid = np.linspace(0, T, n_steps)
brownian_increments = np.random.normal(0, np.sqrt(dt), size=(n_simulations, n_steps))
S_paths = S0 + mu * time_grid + sigma * np.cumsum(brownian_increments, axis=1)

# Final prices
S_T = S_paths[:, -1]

# European Call Payoff
payoffs = np.maximum(S_T - K, 0)

# Control Variate: Arithmetic Average
S_avg = S_paths.mean(axis=1)
expected_S_avg = S0 + mu * T / 2

# Covariance and optimal c
covariance = np.cov(payoffs, S_avg)[0, 1]
variance_x = np.var(S_avg)
c_optimal = covariance / variance_x

# Adjusted payoffs using control variates
adjusted_payoffs = payoffs - c_optimal * (S_avg - expected_S_avg)

# Monte Carlo estimates
call_price = np.mean(payoffs)
adjusted_call_price = np.mean(adjusted_payoffs)

# Variance comparison
variance_original = np.var(payoffs)
variance_adjusted = np.var(adjusted_payoffs)

# Confidence interval computation
confidence_level = 0.95
z = norm.ppf(0.5 + confidence_level / 2)

# Standard errors
se_original = np.std(payoffs) / np.sqrt(n_simulations)
se_adjusted = np.std(adjusted_payoffs) / np.sqrt(n_simulations)

print(se_original, se_adjusted)

# Confidence intervals
ci_original = (call_price - z * se_original, call_price + z * se_original)
ci_adjusted = (adjusted_call_price - z * se_adjusted, adjusted_call_price + z * se_adjusted)

# Results
print(f"Original Call Price Estimate: {call_price:.2f}")
print("Standard Error (Original):", np.round(se_original, 2))
print(f"Adjusted Call Price Estimate (Control Variate): {adjusted_call_price:.2f}")
print("Standard Error (Adjusted):", np.round(se_adjusted, 2))
print(f"Variance Reduction: {100 * (1 - variance_adjusted / variance_original):.2f}%")

# Plot histogram
plt.hist(payoffs, bins=50, alpha=0.5, label="Original Payoffs")
plt.hist(adjusted_payoffs, bins=50, alpha=0.5, label="Adjusted Payoffs (Control Variate)")
plt.axvline(call_price, color='blue', linestyle='--', label="Original Mean")
plt.axvline(adjusted_call_price, color='red', linestyle='--', label="Adjusted Mean")
plt.title("Payoffs with and without Control Variates")
plt.xlabel("Payoff")
plt.ylabel("Frequency")
plt.legend()
plt.show()